In [37]:
import numpy as np

def load_data(second_level_file, first_level_file, items_file):
    """
    Load a two-level bin packing problem instance from .txt files.

    :param second_level_file: path to the level2.txt file (ids and capacities)
    :param first_level_file: path to the level1.txt file (ids and capacities)
    :param items_file: path to the items.txt file (ids and capacities)
    :return: a dictionary containing instance data
    """

    # Load items.txt
    items = []
    with open(items_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            item_id = int(parts[0])
            item_size = float(parts[1])
            items.append((item_id, item_size))
    
    # Load level1.txt
    first_level_types = []
    with open(first_level_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            bin_type = int(parts[0])
            bin_size = float(parts[1])
            first_level_types.append((bin_type, bin_size))
    
    # Load level2.txt
    second_level_types = []
    with open(second_level_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            bin_size = float(parts[1])
            second_level_types.append(bin_size)
    

    instance = {
        "items": np.array(items),
        "first_level_types": np.array(first_level_types),
        "second_level_types": np.array(second_level_types)
    }
    
    return instance


In [38]:
import loader as loader
import numpy as np


def main():

    # Paths to the data files.
    # the data of the first column is the ids, while the second column is the capacities
    second_level_file = "level2.txt" 
    first_level_file = "level1.txt" 
    item_file = "items.txt"  

    # Loading the 2-levels BPP data
    instance = loader.load_data(second_level_file, first_level_file, item_file)
    
    # TODO - Output the solutions of best-fit / first-fit heuristic.


def best_fit_heuristic(instance):

    """
    Algorithm for the best-fit heuristic to solve the Two-level Bin Packing Problem

    :param instance: A dictionary containing the items and bin types.
    :return: List of second-level bins.
    """

    # TODO - Implement the best-fit Heuristic to generate BPP solutions.

def first_fit_heuristic(instance):
    """
    Algorithm for the first-fit heuristic to solve the Two-level Bin Packing Problem.

    :param instance: A dictionary containing the items and bin types.
    :return: List of second-level bins.
    """

    # TODO - Implement the first-fit Heuristic to generate BPP solutions.


if __name__ == '__main__':
    main()


In [39]:
# Load data directly (this creates `instance` in the global notebook scope)
instance = load_data("level2.txt", "level1.txt", "items.txt")


In [40]:
items = instance["items"]
k = len(items) // 4  # should be 10
groups = [items[i*k:(i+1)*k] for i in range(4)]
first_level_capacities = instance["first_level_types"][:, 1]  # c1 to c4
second_level_capacity = instance["second_level_types"][0]  # C


In [41]:
def pack_items_first_fit(items, bin_capacity):
    """
    Packs items into bins using the First-Fit heuristic.

    :param items: List of (id, size) tuples
    :param bin_capacity: Capacity of the current bin type
    :return: List of bins, each a list of item sizes
    """
    bins = []
    for _, size in items:
        placed = False
        for b in bins:
            if sum(b) + size <= bin_capacity:
                b.append(size)
                placed = True
                break
        if not placed:
            bins.append([size])
    return bins


In [42]:
def first_fit_heuristic(instance):
    items = instance["items"]
    k = len(items) // 4
    groups = [items[i * k:(i + 1) * k] for i in range(4)]

    first_level_capacities = instance["first_level_types"][:, 1]
    second_level_capacity = instance["second_level_types"][0]

    all_small_bins = []

    print("Level 1 - Small Bin Packing (First-Fit):\n")

    for i, group in enumerate(groups):
        cap = first_level_capacities[i]
        packed_bins = pack_items_first_fit(group, cap)

        print(f"Bin Type {i+1} (capacity {cap}):")
        for j, b in enumerate(packed_bins):
            print(f"  S{i+1}_{j+1}: {', '.join(f'{x:.2f}' for x in b)}")
            all_small_bins.append((cap, b))  # store capacity for Level 2

        print(f"  Total number of bins: {len(packed_bins)}\n")

    # Level 2 - Pack small bins into large bins
    large_bins = []
    for cap, _ in all_small_bins:
        placed = False
        for lb in large_bins:
            if sum(lb) + cap <= second_level_capacity:
                lb.append(cap)
                placed = True
                break
        if not placed:
            large_bins.append([cap])

    print("Level 2 - Large Bin Packing (First-Fit):\n")
    for i, lb in enumerate(large_bins):
        print(f"  L{i+1}: {', '.join(f'{x:.2f}' for x in lb)}")
    print(f"  Total number of bins: {len(large_bins)}")


In [43]:
first_fit_heuristic(instance)


Level 1 - Small Bin Packing (First-Fit):

Bin Type 1 (capacity 3.0):
  S1_1: 1.12, 1.80
  S1_2: 2.85
  S1_3: 2.20, 0.47, 0.17
  S1_4: 2.47
  S1_5: 0.60, 0.90
  S1_6: 2.12
  Total number of bins: 6

Bin Type 2 (capacity 1.0):
  S2_1: 0.02, 0.97
  S2_2: 0.83
  S2_3: 0.21, 0.35, 0.28
  S2_4: 0.30, 0.52
  S2_5: 0.43, 0.29
  Total number of bins: 5

Bin Type 3 (capacity 4.0):
  S3_1: 2.45, 0.56, 0.80
  S3_2: 1.17, 1.47, 1.19
  S3_3: 1.82, 2.06
  S3_4: 3.14
  S3_5: 2.37
  Total number of bins: 5

Bin Type 4 (capacity 2.0):
  S4_1: 1.22, 0.34, 0.13, 0.20
  S4_2: 1.90
  S4_3: 1.93
  S4_4: 1.62
  S4_5: 0.61, 1.37
  S4_6: 0.88
  Total number of bins: 6

Level 2 - Large Bin Packing (First-Fit):

  L1: 3.00, 1.00, 1.00
  L2: 3.00, 1.00, 1.00
  L3: 3.00, 1.00
  L4: 3.00, 2.00
  L5: 3.00, 2.00
  L6: 3.00, 2.00
  L7: 4.00
  L8: 4.00
  L9: 4.00
  L10: 4.00
  L11: 4.00
  L12: 2.00, 2.00
  L13: 2.00
  Total number of bins: 13


In [44]:
def pack_items_best_fit(items, bin_capacity):
    """
    Packs items into bins using the Best-Fit heuristic.

    :param items: List of (id, size) tuples
    :param bin_capacity: Capacity of the current bin type
    :return: List of bins, each a list of item sizes
    """
    bins = []
    for _, size in items:
        best_bin_index = -1
        min_space_left = bin_capacity + 1  # Start with something bigger than any possible bin

        for i, b in enumerate(bins):
            space_left = bin_capacity - sum(b)
            if space_left >= size and space_left - size < min_space_left:
                best_bin_index = i
                min_space_left = space_left - size

        if best_bin_index != -1:
            bins[best_bin_index].append(size)
        else:
            bins.append([size])
    return bins


In [45]:
def best_fit_heuristic(instance):
    items = instance["items"]
    k = len(items) // 4
    groups = [items[i * k:(i + 1) * k] for i in range(4)]

    first_level_capacities = instance["first_level_types"][:, 1]
    second_level_capacity = instance["second_level_types"][0]

    all_small_bins = []

    print("Level 1 - Small Bin Packing (Best-Fit):\n")

    for i, group in enumerate(groups):
        cap = first_level_capacities[i]
        packed_bins = pack_items_best_fit(group, cap)

        print(f"Bin Type {i+1} (capacity {cap}):")
        for j, b in enumerate(packed_bins):
            print(f"  S{i+1}_{j+1}: {', '.join(f'{x:.2f}' for x in b)}")
            all_small_bins.append((cap, b))  # store capacity for Level 2

        print(f"  Total number of bins: {len(packed_bins)}\n")

    # Level 2 - Pack small bins into large bins using Best-Fit
    large_bins = []
    for cap, _ in all_small_bins:
        best_index = -1
        min_space_left = second_level_capacity + 1

        for i, lb in enumerate(large_bins):
            space_left = second_level_capacity - sum(lb)
            if space_left >= cap and space_left - cap < min_space_left:
                best_index = i
                min_space_left = space_left - cap

        if best_index != -1:
            large_bins[best_index].append(cap)
        else:
            large_bins.append([cap])

    print("Level 2 - Large Bin Packing (Best-Fit):\n")
    for i, lb in enumerate(large_bins):
        print(f"  L{i+1}: {', '.join(f'{x:.2f}' for x in lb)}")
    print(f"  Total number of bins: {len(large_bins)}")


In [46]:
best_fit_heuristic(instance)


Level 1 - Small Bin Packing (Best-Fit):

Bin Type 1 (capacity 3.0):
  S1_1: 1.12, 1.80
  S1_2: 2.85
  S1_3: 2.20, 0.47, 0.17
  S1_4: 2.47
  S1_5: 0.60, 0.90
  S1_6: 2.12
  Total number of bins: 6

Bin Type 2 (capacity 1.0):
  S2_1: 0.02, 0.97
  S2_2: 0.83
  S2_3: 0.21, 0.35, 0.28
  S2_4: 0.30, 0.52
  S2_5: 0.43, 0.29
  Total number of bins: 5

Bin Type 3 (capacity 4.0):
  S3_1: 2.45, 0.56
  S3_2: 1.17, 1.47, 1.19
  S3_3: 1.82, 2.06
  S3_4: 3.14, 0.80
  S3_5: 2.37
  Total number of bins: 5

Bin Type 4 (capacity 2.0):
  S4_1: 1.22, 0.34, 0.13, 0.20
  S4_2: 1.90
  S4_3: 1.93
  S4_4: 1.62
  S4_5: 0.61, 1.37
  S4_6: 0.88
  Total number of bins: 6

Level 2 - Large Bin Packing (Best-Fit):

  L1: 3.00, 1.00, 1.00
  L2: 3.00, 1.00, 1.00
  L3: 3.00, 1.00
  L4: 3.00, 2.00
  L5: 3.00, 2.00
  L6: 3.00, 2.00
  L7: 4.00
  L8: 4.00
  L9: 4.00
  L10: 4.00
  L11: 4.00
  L12: 2.00, 2.00
  L13: 2.00
  Total number of bins: 13
